In [2]:
import os

Set up paths in cell below

In [5]:
collect_date = '20220407'
homedir ='/Users/rdcrlsro/Documents/Projects/SnowEx_helipod/2022/20220407/MCS/'
jason_path = os.path.join(homedir,'jason')
input_path = os.path.join(homedir,'data')
proc_path = os.path.join(homedir,'data/*.laz')
output_path = os.path.join(homedir,'out/PC/')
tile_path = os.path.join(output_path,'tiles')
mosaic_path = os.path.join(output_path, 'Mosaic')


In [ ]:
os.chdir(jason_path)
!cat Fix_CRS.json

below i pass paths as variables into parallel and then into pdal. parallel (https://www.gnu.org/software/parallel/sphinx.html) reads all files from input_path and then writes to the the relative path. This solves batching issues, for this step anyway

In [3]:
#assign CRS (32611) to each point
#parallel holds file names in {}. {/.} appends the file name with cleanCRS 
os.chdir(input_path)

!ls $input_path | parallel -I{} \
    pdal pipeline ./Fix_CRS.json \
    --readers.las.filename={} \
    --writers.las.filename=../out/PC/cleanCRS{/.}.las 

PDAL: readers.las: Couldn't read LAS header. File size insufficient.



Create a mosaic of all flight lines for the collect using pdal merge. Specifying a unique file name may require use of filters.merge and building/ammending a json file with all .laz input files

In [47]:

#this simple method is now working, and producing the correct output, but no idea how to pass a variable as a output file name
#if file name is left non-specfic (no date), then its easier to keep this batchy without modifications to paths per mission.
os.chdir(output_path)
!pdal merge ./*.las Mosaic.laz

In [14]:
#find grabs full path rather than just file name like ls. relative paths are working in parallel call, but not always in --options for pdal. 
#MC is mosaic, cleaned (DEM and outlier filters), 
os.chdir(mosaic_path)
!find *.laz | \
parallel -I{} pdal pipeline ../../../jason/cleanPC.json \
--readers.las.filename={} \
--writers.las.filename=MC{/.}.laz \

The smrf filter works only if returns are >=1. 0 value stops the process.  there are 105 0 value points in the cloud.  Need to learn to use filters.mongo to write an expression to remove those points.

In [31]:
os.chdir(mosaic_path)
os.chdir("clean")

!find *.laz | \
parallel -I{} pdal pipeline ../../../../jason/remove_veg.json \
--readers.las.filename={} \
--writers.las.filename=Vo{/.}.las \
--writers.gdal.filename=Vo{/.}.tif 